In [7]:
import praw
import datetime
import time
import os
filepath = './TerraceHouse/'

In [8]:
# client_id and client_secret should be your own 
reddit = praw.Reddit(client_id='<insert-client-id>',
                     client_secret='<insert-client-secret>',
                     user_agent='my user agent')

In [9]:
def write_to_file(filepath, filename, comment_list):
    # Writing to file
    record_file = open(filepath + str(filename), 'a')
    
    for comment in comment_list:
        record_file.write(comment[0] + '\t')
        record_file.write(comment[1] + '\t')
        record_file.write(comment[2] + '\t')
        record_file.write(comment[3] + '\t')
        record_file.write(comment[4] + '\t')
        record_file.write(comment[5])
        record_file.write('\n')
    
    record_file.close()

In [10]:
def process_comments(title, objects, comment_hierarchy):    
    for object in objects:
        if type(object).__name__ == "Comment":
            comment = []
            
            comment_date = int(object.created)
            
            # append values that we want into the comment list, each list is an 'object'
            comment.append(title)
            comment.append('level ' + str(comment_hierarchy))
            comment.append(str(object.author))
            comment.append(str(object.score) + ' points')
            comment.append(str(object.body).replace('\n', ''))
            comment.append(datetime.datetime.fromtimestamp(comment_date).strftime('%Y-%m-%d'))
            
            # append a completed comment into the comment_list, declared in the submission loop 
            comment_list.append(comment)
                        
            if(object.replies.__len__() >= 0):
                process_comments(title, object.replies, comment_hierarchy + 1) # Get replies of comment
      
        elif type(object).__name__ == "MoreComments":
            process_comments(title, object.comments(), 1) # Get more comments at top level

In [11]:
# submission = reddit.submission(id='91317x')

subreddit = reddit.subreddit('terracehouse')
search_terms = ['hana']

for term in search_terms:
    filename = term + '.tsv'
    record_file = open(filepath + filename, 'a')
    record_file.write('initial_title\tcomment_hierarchy\tuserhandle\tcomment_popularity\tuser_post\tcomment_date\n')
    record_file.close()
    for submission in subreddit.search(term):
        
        comment_list = []
        process_comments(submission.title, submission.comments, 1)
        
        # every search term will create a tsv with the same name
        write_to_file(filepath, filename, comment_list)
        
        time.sleep(.500) # sleep 500ms to prevent flooding the api endpoint
